In [14]:
import torch
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

import os 
import numpy as np
import pandas as pd

import glob
import random

from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout,AdaptiveAvgPool2d
from torch.optim import Adam, SGD
from torch.utils.data import DataLoader
import sklearn

In [15]:
#Hyper parameters
test_datapath='/content/drive/MyDrive/1creditproject/test.npy'#Give input file
test_labelpath='/content/drive/MyDrive/1creditproject/label_test.npy'
modelpath='/content/drive/MyDrive/1creditproject/models/' #Load trained model
classes=10
epochs=50




In [16]:
test_data=np.load(test_datapath)
test_label=np.load(test_labelpath)

In [17]:
print(test_data.shape)

(2518, 40, 501)


In [22]:
#set device
# Device will determine whether to run the training on GPU or CPU.
cuda0 = torch.device('cuda:0')
device = cuda0

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



In [23]:
print(torch.device('cuda:0'))

cuda:0


In [24]:
#CNN model
class CNN(nn.Module):
    def __init__(self,classes):
        super(CNN,self).__init__()
        self.relu = nn.ReLU()
        self.max1=MaxPool2d(kernel_size=(5,5)) 
        self.max2=MaxPool2d(kernel_size=(4,90)) 
        self.drop=Dropout(0.3)
        self.conv1=Conv2d(in_channels=1,out_channels=32,kernel_size=(7,7),padding=(2,2))
        self.b1=BatchNorm2d(32)
        self.conv2=Conv2d(in_channels=32,out_channels=64,kernel_size=(7,7),padding=(2,2))
        self.b2=BatchNorm2d(64)
        self.gap = AdaptiveAvgPool2d((1,1))
        self.lin1=Linear(64,100)
        self.lin2=Linear(100,10)
    def forward(self,x):
       # print(x.shape)
        x=(self.relu(self.b1(self.conv1(x))))
        x=self.max1(x)
        x=self.drop(x)
        x=(self.relu(self.b2(self.conv2(x))))
        x=self.max2(x)
        x=self.drop(x)
        x=self.gap(x)
#         print(x.shape)
        x = x.view(x.shape[0],-1)
#         print(x.shape)
        x=self.lin1(x)
        
        x=self.lin2(x)
        
        return x
        
           
        




In [40]:
accuracy=[]
for e in range(epochs):
    model=CNN(classes)
    path = modelpath+str(e+1)+".pth"
    print(path)
    model.load_state_dict(torch.load(path))
    model.to(device)
    model.double()
    manual_seed = random.randint(1,10000)
    random.seed(manual_seed)
    torch.manual_seed(manual_seed)
    
    #random.shuffle(test_loader)
    true=[]
    pred=[]
    
    for i in range(len(test_data)):
        t1=test_label[i]
        td=test_data[i]
        test_file=torch.from_numpy(td)
        test_file=torch.unsqueeze(test_file,0)
        test_file=torch.unsqueeze(test_file,0)
        test_file.to(device)
        prediction = model(test_file.cuda())
        #p=prediction.detach().numpy()
        p1=torch.argmax(prediction)
        true.append(t1)
        pred.append(p1)
    length=len(true)
    correct=0
    for i in range(len(true)):
        if true[i]==pred[i]:
            correct+=1
    val=float(correct*100/length)
    accuracy.append(val)
    print("Accuracy of model "+str(e+1)+" is :",val)
max_val=max(accuracy)
print("Max accuracy occured at epoch "+str(accuracy.index(max_val)+1)+" : "+str(max_val))
        

/content/drive/MyDrive/1creditproject/models/1.pth
Accuracy of model 1 is : 14.535345512311359
/content/drive/MyDrive/1creditproject/models/2.pth
Accuracy of model 2 is : 20.293884034948373
/content/drive/MyDrive/1creditproject/models/3.pth
Accuracy of model 3 is : 27.44241461477363
/content/drive/MyDrive/1creditproject/models/4.pth
Accuracy of model 4 is : 33.915806195393166
/content/drive/MyDrive/1creditproject/models/5.pth
Accuracy of model 5 is : 31.8109610802224
/content/drive/MyDrive/1creditproject/models/6.pth
Accuracy of model 6 is : 31.890389197776013
/content/drive/MyDrive/1creditproject/models/7.pth
Accuracy of model 7 is : 32.80381254964257
/content/drive/MyDrive/1creditproject/models/8.pth
Accuracy of model 8 is : 36.3780778395552
/content/drive/MyDrive/1creditproject/models/9.pth
Accuracy of model 9 is : 38.00635424940429
/content/drive/MyDrive/1creditproject/models/10.pth
Accuracy of model 10 is : 38.0857823669579
/content/drive/MyDrive/1creditproject/models/11.pth
Accur